In [ ]:
import os
import shutil
import re
import subprocess
from datetime import datetime
import pandas as pd

# --- CONFIGURATION ---

# Set to 1 to copy data from iCloud before committing. 
# Set to 0 to skip the copy and only do the git commit+push.
SYNC_FROM_ICLOUD = 0

SOURCE_BASE_DIR = '/Users/eyeq/Library/Mobile Documents/com~apple~CloudDocs/EyeQ'

# Assumes this script is run from the /MyStudyDashboard root,
# and the data goes into the existing /EyeQ subdirectory.
DEST_BASE_DIR = './EyeQ'

#Git Settings (Used for the commit message)
COMMIT_MESSAGE = "AUTO-SYNC: Updated data from local study folder."
GIT_BRANCH = "main" # The branch to push to


def sync_data():
    """
    Scans the source directory for PID/final folders, copies them selectively
    to the destination directory, and then commits the changes to Git.
    """

    print("--- Starting Data Synchronization ---")

    # Ensure the destination data directory exists
    os.makedirs(DEST_BASE_DIR, exist_ok=True)

    
    # PART 1: Copying Files from iCloud
    pids_synced = []

    if SYNC_FROM_ICLOUD:
        excluded_PIDs = pd.read_excel('../exclusion_tracker.xlsx')
        excluded_PIDs = excluded_PIDs[excluded_PIDs['Exclude'] == 1]
        dq_PIDs = excluded_PIDs['PID'].tolist()

        try:
            # Get all items in the Source Base Directory
            for item in os.listdir(SOURCE_BASE_DIR):
                # Check if the item name is a number (a PID folder)
                if re.fullmatch(r'\d+', item):
                    pid = item

                    if int(pid) not in dq_PIDs:
                        source_pid_dir = os.path.join(SOURCE_BASE_DIR, pid)
                        source_final_dir = os.path.join(source_pid_dir, 'final')

                        # Check if the required 'final' directory exists in the source PID folder
                        if os.path.isdir(source_final_dir):
                            dest_pid_dir = os.path.join(DEST_BASE_DIR, pid)
                            dest_final_dir = os.path.join(dest_pid_dir, 'final')

                            # Create the destination structure if it doesn't exist
                            os.makedirs(dest_final_dir, exist_ok=True)

                            # Copy files from source/PID/final to dest/PID/final
                            for filename in os.listdir(source_final_dir):
                                src_file = os.path.join(source_final_dir, filename)
                                dst_file = os.path.join(dest_final_dir, filename)

                                # Use shutil.copy2 to copy the file, preserving metadata (timestamps)
                                if os.path.isfile(src_file):
                                    # Check if the file already exists at the destination
                                    if os.path.exists(dst_file):
                                        # Only copy if the source file is newer (to avoid unnecessary disk writes)
                                        if os.path.getmtime(src_file) > os.path.getmtime(dst_file):
                                            shutil.copy2(src_file, dst_file)
                                            print(f"   [UPDATED] {pid}/{filename}")
                                    else:
                                        # File does not exist, copy it
                                        shutil.copy2(src_file, dst_file)
                                        print(f"   [NEW FILE] {pid}/{filename}")

                            pids_synced.append(pid)
                    else:
                        print(f'Skipping {pid}')

        except FileNotFoundError as e:
            print(f"\n[ERROR] Source directory not found: {e}")
            print("Please verify the path: '/Users/eyeq/Library/Mobile Documents/com~apple~CloudDocs/EyeQ'")
            return
        except Exception as e:
            print(f"\n[CRITICAL ERROR] Failed during file synchronization: {e}")
            return

        if not pids_synced:
            print("\nNo new or updated PID data found. Skipping Git operations.")
            return

        print(f"\nSuccessfully synced data for {len(pids_synced)} PIDs.")
    else:
        print("iCloud sync disabled (SYNC_FROM_ICLOUD = 0). Skipping file copy.")

    # PART 2: Git Push
    print("\n--- Starting Git Commit & Push ---")

    try:
        # Stage all changes in the current directory (recursively adds new data)
        print("1. Staging changes...")
        subprocess.run(['git', 'add', DEST_BASE_DIR], check=True, capture_output=True, text=True)

        # Commit changes with a timestamp
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        commit_msg = f"{COMMIT_MESSAGE} ({timestamp})"
        print(f"2. Committing with message: {commit_msg}")
        subprocess.run(['git', 'commit', '-m', commit_msg], check=True, capture_output=True, text=True)

        # 3. Push to the remote repository (will use your SSH key)
        print(f"3. Pushing to '{GIT_BRANCH}' branch...")
        push_result = subprocess.run(['git', 'push', 'origin', GIT_BRANCH], check=True, capture_output=True, text=True)

        print("\n--- Synchronization COMPLETE ---")
        print("Data is committed and pushed to GitHub. Render should automatically deploy.")

    except subprocess.CalledProcessError as e:
        print("\n[GIT ERROR] Failed to execute Git command.")
        print(f"Command failed: {e.cmd}")
        print(f"Error output:\n{e.stderr}")
        print("\nEnsure your SSH key is running (run `ssh-add` if necessary) and the current folder is a Git repository.")
    except Exception as e:
        print(f"\n[CRITICAL ERROR] An unexpected error occurred during Git operations: {e}")


if __name__ == "__main__":
    sync_data()